# Resampling Techniques - Homework 1

### Import Necessary Libraries

In [93]:
# general
from tqdm import tqdm
import itertools


# matrix manipulation
import numpy as np

# plotting
import matplotlib.pyplot as plt

# custom probability distribution
from scipy import stats
from scipy.stats import norm, cauchy
from scipy.special import ndtr # standard normal cdf
from statsmodels.distributions.empirical_distribution import ECDF # empirical cdf

#### Parameter Estimate

Random variable with custom CDF or PDF is taken care of by the following 2 classes. For entropy calculation rather than integrating, Monte Carlo Estimation is used to ensure faster computation, but it results in poor estimates and hence low coverage probabilities as can be seen in all the simulations.

$$Entropy_{MC} = -\frac{1}{n}\sum_{i=1}^{n}\log{\hat{f}(X_i)}$$
where, $\{X_i\}_{i=1}^n$ is a random sample from the CDF $F$ or empirical CDF $F_n$ and since empirical pdf for continuous distribution is almost impossible to find out as each value needs to repeat in that case which has a very low probability and theoretically $0$. I use histogram as the estimate for the empirical pdf and is denoted by $\hat{f}$.

In [2]:
# RV with custom cdf
class CustomCDFRV(stats.rv_continuous):
    def __init__(self, cdf_func, a = None, b = None, xtol = 1e-14, seed = None):
        super().__init__(a = a, b = b, xtol=xtol, seed=seed)
        self.cdf_func = cdf_func

    def _cdf(self, x):
        return self.cdf_func(x)
    
    def monte_mean(self, num_monte_carlo:int = 100): # it is faster than finding mean by integration
        return np.mean(self.rvs(size=num_monte_carlo))
    
    def _entropy(self, num_monte_carlo:int = 100): # doing integration gives unstable result
        return np.mean(-np.log(self.pdf(self.rvs(size=num_monte_carlo))))

# RV with custom pdf -- IT'S SLOW AND ERRANEOUS
class CustomPDFRV(stats.rv_continuous):
    def __init__(self, pdf_func, a = None, b = None, xtol = 1e-14, seed = None):
        super().__init__(a=a, b = b, xtol=xtol, seed=seed)
        self.pdf_func = pdf_func

    def _pdf(self, x):
        return self.pdf_func(x)
    
    def monte_mean(self, num_monte_carlo:int = 100): # it is faster than finding mean by integration
        return np.mean(self.rvs(size=num_monte_carlo))
    
    def _entropy(self, num_monte_carlo:int = 100): # doing integration gives unstable result
        return np.mean(-np.log(self.pdf(self.rvs(size=num_monte_carlo))))

Since, estimating differential entropy requires estimate for the empirical pdf. I use the histogram here as the estimate for the empirical pdf. The binwidth is chosen using the following thumb rule,
$$h = 2\times IQR \times n^{-\frac{1}{3}}$$

In [3]:
def emp_entropy(ecdf):
    iqr = np.quantile(ecdf.cdf_func.x[1:], 3/4, method='lower') - np.quantile(ecdf.cdf_func.x[1:], 1/4, method='lower')
    h = 2 * iqr * np.float_power(len(ecdf.cdf_func.x[1:]), -1/3)
    num_bins = int((np.max(ecdf.cdf_func.x[1:]) - np.min(ecdf.cdf_func.x[1:])) // h)
    probs, bin_edges = np.histogram(ecdf.cdf_func.x[1:], density=True, bins=num_bins)
    bin_edges =  np.concatenate([[-np.inf], bin_edges, [np.inf]])
    probs = np.concatenate([[0.0], probs, [0.0]])
    return np.ma.masked_invalid(-np.log([probs[np.argmax(bin_edges >= x) - 1] for x in ecdf.cdf_func.x[1:]])).mean()

The following are some of the values chosen for the rest of the assignment.

In [4]:
SAMPLE_SIZE = 20
NUM_BOOTSTRAPS = 50
COVERAGE_PROB_ITERS = 100

In [5]:
cust_norm_rv = CustomCDFRV(ndtr) # standard normal random variable
cust_exp_rv = CustomCDFRV(lambda x: 1-np.exp(-x), a = 0) # Exp(1) random variable

In [6]:
cust_norm_rv.entropy()

1.4629070222344713

#### Exact Resampling

Here, I resampled the sample again and again from the true distribution function and treated them as Bootstrap sample. The results are obviously much better than the other techniques as it relies on having access to the true distribution as many number of times as wished. The confidence interval has comparitively smaller length and the coverage probabilities are near perfect.

In [7]:
def exact_resampling(RV, statistic, sample_size: int = SAMPLE_SIZE, num_bootstraps: int = NUM_BOOTSTRAPS, **kwargs):

    # gets the resampled data
    resampled_data = np.array([RV.rvs(size=sample_size, **kwargs) for _ in range(num_bootstraps)])

    # gets the ecdf for the entire resampled data
    net_ecdf = CustomCDFRV(ECDF(resampled_data.ravel()), np.min(resampled_data), np.max(resampled_data))

    # get point estimate
    point_estimate = statistic(net_ecdf)

    # gets the ECDF of each resampled data
    ecdf_data = [CustomCDFRV(ECDF(bootstrap_sample), np.min(bootstrap_sample), np.max(bootstrap_sample)) for bootstrap_sample in resampled_data]

    # gets the resample data statistic
    resampled_statistics = np.array([np.sqrt(sample_size) * (statistic(ecdf) - point_estimate) for ecdf in ecdf_data])

    # gets the resampled data statistic's ecdf
    resampled_statistic_ecdf = CustomCDFRV(ECDF(resampled_statistics))

    return np.array([point_estimate + resampled_statistic_ecdf.ppf(0.025)/np.sqrt(sample_size), point_estimate + 
            resampled_statistic_ecdf.ppf(0.975)/np.sqrt(sample_size)])

In [20]:
def evaluate_all(RV, **kwargs):
    print('*** BOOTSTRAP CONFIDENCE INTERVALS ***')
    print('For Mean:', exact_resampling(RV, lambda ecdf: ecdf.cdf_func.x[1:].mean(), **kwargs))
    print('For Median:', exact_resampling(RV, lambda ecdf: ecdf.median(), **kwargs))
    print('For IQR:', exact_resampling(RV, lambda ecdf: ecdf.ppf(3/4) - ecdf.ppf(1/4), **kwargs))
    print('For Entropy:', exact_resampling(RV, emp_entropy, **kwargs))

    print()
    print('*** TRUE PARAMETER VALUES ***')
    print('For Mean:', RV.mean(**kwargs))
    print('For Median:', RV.median(**kwargs))
    print('For IQR:', RV.ppf(3/4, **kwargs) - RV.ppf(1/4, **kwargs))
    print('For Entropy:', RV.entropy(**kwargs))

    print()
    print('*** COVERAGE PROBABILITIES ***')
    print('For Mean:', np.mean([np.sum(exact_resampling(RV, lambda ecdf: ecdf.cdf_func.x[1:].mean(), **kwargs) <= RV.mean(**kwargs)) == 1 for _ in tqdm(range(COVERAGE_PROB_ITERS))]))
    print('For Median:', np.mean([np.sum(exact_resampling(RV, lambda ecdf: ecdf.median(), **kwargs) <= RV.median(**kwargs)) == 1 for _ in tqdm(range(COVERAGE_PROB_ITERS))]))
    print('For IQR:', np.mean([np.sum(exact_resampling(RV, lambda ecdf: ecdf.ppf(3/4) - ecdf.ppf(1/4), **kwargs) <= RV.ppf(3/4, **kwargs) - RV.ppf(1/4, **kwargs)) == 1 for _ in tqdm(range(COVERAGE_PROB_ITERS))]))
    print('For Entropy:', np.mean([np.sum(exact_resampling(RV, emp_entropy, **kwargs) <= RV.entropy(**kwargs)) == 1 for _ in tqdm(range(COVERAGE_PROB_ITERS))]))

##### Normal(0, 1) Distribution

In [21]:
evaluate_all(norm)

*** BOOTSTRAP CONFIDENCE INTERVALS ***
For Mean: [-0.50162051  0.46058489]
For Median: [-0.44724029  0.48643302]
For IQR: [0.85025879 1.92980969]


/var/folders/mg/l90d6s9536n6mfzjbqgrxffm0000gn/T/ipykernel_14340/184846182.py:8: RuntimeWarning: divide by zero encountered in log
  return np.ma.masked_invalid(-np.log([probs[np.argmax(bin_edges >= x) - 1] for x in ecdf.cdf_func.x[1:]])).mean()


For Entropy: [0.90920166 1.42997591]

*** TRUE PARAMETER VALUES ***
For Mean: 0.0
For Median: 0.0
For IQR: 1.3489795003921634
For Entropy: 1.4189385332046727

*** COVERAGE PROBABILITIES ***


100%|██████████| 100/100 [00:01<00:00, 56.29it/s]


For Mean: 1.0


...

For Median: 1.0


100%|██████████| 100/100 [00:03<00:00, 25.64it/s]


For IQR: 1.0


100%|██████████| 100/100 [00:02<00:00, 37.36it/s]

For Entropy: 0.92


##### Normal(5, 1) Distribution

In [22]:
evaluate_all(norm, loc=5)

/var/folders/mg/l90d6s9536n6mfzjbqgrxffm0000gn/T/ipykernel_14340/184846182.py:8: RuntimeWarning: divide by zero encountered in log
  return np.ma.masked_invalid(-np.log([probs[np.argmax(bin_edges >= x) - 1] for x in ecdf.cdf_func.x[1:]])).mean()


*** BOOTSTRAP CONFIDENCE INTERVALS ***
For Mean: [4.53342066 5.34385121]
For Median: [4.66192238 5.54904764]
For IQR: [0.78117713 1.9739037 ]
For Entropy: [0.81874155 1.4221989 ]

*** TRUE PARAMETER VALUES ***
For Mean: 5.0
For Median: 5.0
For IQR: 1.3489795003921632
For Entropy: 1.4189385332046727

*** COVERAGE PROBABILITIES ***


100%|██████████| 100/100 [00:01<00:00, 55.80it/s]


For Mean: 1.0


100%|██████████| 100/100 [00:12<00:00,  7.76it/s]


For Median: 1.0


100%|██████████| 100/100 [00:03<00:00, 25.21it/s]


For IQR: 1.0


100%|██████████| 100/100 [00:02<00:00, 37.67it/s]

For Entropy: 0.89


##### Normal(5, 9) Distribution

In [23]:
evaluate_all(norm, loc=5, scale=3)

/var/folders/mg/l90d6s9536n6mfzjbqgrxffm0000gn/T/ipykernel_14340/184846182.py:8: RuntimeWarning: divide by zero encountered in log
  return np.ma.masked_invalid(-np.log([probs[np.argmax(bin_edges >= x) - 1] for x in ecdf.cdf_func.x[1:]])).mean()


*** BOOTSTRAP CONFIDENCE INTERVALS ***
For Mean: [3.64363594 6.30702352]
For Median: [3.89787655 7.07393481]
For IQR: [2.08583044 5.20921586]
For Entropy: [1.86699938 2.61216594]

*** TRUE PARAMETER VALUES ***
For Mean: 5.0
For Median: 5.0
For IQR: 4.046938501176491
For Entropy: 2.5175508218727822

*** COVERAGE PROBABILITIES ***


100%|██████████| 100/100 [00:01<00:00, 56.04it/s]


For Mean: 1.0


100%|██████████| 100/100 [00:13<00:00,  7.51it/s]


For Median: 1.0


100%|██████████| 100/100 [00:03<00:00, 25.41it/s]


For IQR: 1.0


100%|██████████| 100/100 [00:02<00:00, 37.11it/s]

For Entropy: 0.9


##### Cauchy(5, 9) Distribution

In [24]:
evaluate_all(cauchy, loc=5, scale=3)

*** BOOTSTRAP CONFIDENCE INTERVALS ***
For Mean: [-25.43186907  44.29733189]
For Median: [3.05560127 8.69550191]
For IQR: [ 3.28674209 14.15490763]


/var/folders/mg/l90d6s9536n6mfzjbqgrxffm0000gn/T/ipykernel_14340/184846182.py:8: RuntimeWarning: divide by zero encountered in log
  return np.ma.masked_invalid(-np.log([probs[np.argmax(bin_edges >= x) - 1] for x in ecdf.cdf_func.x[1:]])).mean()


For Entropy: [2.35118421 3.84739899]

*** TRUE PARAMETER VALUES ***
For Mean: nan
For Median: 5.0
For IQR: 6.0
For Entropy: 3.6296365356374007

*** COVERAGE PROBABILITIES ***


100%|██████████| 100/100 [00:01<00:00, 55.66it/s]


For Mean: 0.0


...

For Median: 1.0


100%|██████████| 100/100 [00:04<00:00, 21.21it/s]


For IQR: 1.0


100%|██████████| 100/100 [00:02<00:00, 36.16it/s]

For Entropy: 0.79


##### 1/2 * Normal(5, 1) + 1/2 * Cauchy(10, 1) Distribution

In [25]:
evaluate_all(CustomCDFRV(lambda x: (norm.cdf(x, loc=5, scale=1) + cauchy.cdf(x, loc=10, scale=1))/2))

*** BOOTSTRAP CONFIDENCE INTERVALS ***
For Mean: [ 3.5957454  14.10338629]
For Median: [5.2445834  9.79953697]
For IQR: [2.11966964 6.47262403]


/var/folders/mg/l90d6s9536n6mfzjbqgrxffm0000gn/T/ipykernel_14340/184846182.py:8: RuntimeWarning: divide by zero encountered in log
  return np.ma.masked_invalid(-np.log([probs[np.argmax(bin_edges >= x) - 1] for x in ecdf.cdf_func.x[1:]])).mean()


For Entropy: [1.86530187 2.69646429]

*** TRUE PARAMETER VALUES ***
For Mean: 7.500000000000184
For Median: 6.368752841046282
For IQR: 5.153524566844071
For Entropy: 2.5698023611170435

*** COVERAGE PROBABILITIES ***


100%|██████████| 100/100 [03:35<00:00,  2.16s/it]


For Mean: 1.0


...

For Median: 1.0


100%|██████████| 100/100 [01:53<00:00,  1.13s/it]


For IQR: 1.0


100%|██████████| 100/100 [02:02<00:00,  1.22s/it]

For Entropy: 0.91


#### Non-Parametric Bootstrap

Here, I took one sample from the true distribution and then using that sample the resampling was done to obtain the bootstrap estimate. I used $F_n$ i.e. empirical CDF as an estimator of $F$.

In [33]:
def nonparam_bootstrapping(RV, statistic, sample_size: int = SAMPLE_SIZE, num_bootstraps: int = NUM_BOOTSTRAPS, **kwargs):

    # get the sample
    sample_data = RV.rvs(size=sample_size, **kwargs)

    # gets the ecdf for the sample
    net_ecdf_rv = CustomCDFRV(ECDF(sample_data), np.min(sample_data), np.max(sample_data))

    # get point estimate
    point_estimate = statistic(net_ecdf_rv)

    # gets the resampled data
    resampled_data = np.array([net_ecdf_rv.rvs(size=sample_size) for _ in range(num_bootstraps)])

    # gets the ECDF of each resampled data
    ecdf_data = [CustomCDFRV(ECDF(bootstrap_sample), np.min(bootstrap_sample), np.max(bootstrap_sample)) for bootstrap_sample in resampled_data]

    # gets the resample data statistic
    resampled_statistics = np.array([np.sqrt(sample_size) * (statistic(ecdf) - point_estimate) for ecdf in ecdf_data])

    # gets the resampled data statistic's ecdf
    resampled_statistic_ecdf = CustomCDFRV(ECDF(resampled_statistics))

    return np.array([point_estimate + resampled_statistic_ecdf.ppf(0.025)/np.sqrt(sample_size), point_estimate + 
            resampled_statistic_ecdf.ppf(0.975)/np.sqrt(sample_size)])

In [34]:
def evaluate_all(RV, **kwargs):
    print('*** BOOTSTRAP CONFIDENCE INTERVALS ***')
    print('For Mean:', nonparam_bootstrapping(RV, lambda ecdf: ecdf.cdf_func.x[1:].mean(), **kwargs))
    print('For Median:', nonparam_bootstrapping(RV, lambda ecdf: ecdf.median(), **kwargs))
    print('For IQR:', nonparam_bootstrapping(RV, lambda ecdf: ecdf.ppf(3/4) - ecdf.ppf(1/4), **kwargs))
    print('For Entropy:', nonparam_bootstrapping(RV, emp_entropy, **kwargs))

    true_mean = RV.mean(**kwargs)
    true_median = RV.median(**kwargs)
    true_iqr = RV.ppf(3/4, **kwargs) - RV.ppf(1/4, **kwargs)
    true_entropy = RV.entropy(**kwargs)

    print()
    print('*** TRUE PARAMETER VALUES ***')
    print('For Mean:', true_mean)
    print('For Median:', true_median)
    print('For IQR:', true_iqr)
    print('For Entropy:', true_entropy)

    print()
    print('*** COVERAGE PROBABILITIES ***')
    print('For Mean:', np.mean([np.sum(nonparam_bootstrapping(RV, lambda ecdf: ecdf.cdf_func.x[1:].mean(), **kwargs) <= true_mean) == 1 for _ in tqdm(range(COVERAGE_PROB_ITERS))]))
    print('For Median:', np.mean([np.sum(nonparam_bootstrapping(RV, lambda ecdf: ecdf.median(), **kwargs) <= true_median) == 1 for _ in tqdm(range(COVERAGE_PROB_ITERS))]))
    print('For IQR:', np.mean([np.sum(nonparam_bootstrapping(RV, lambda ecdf: ecdf.ppf(3/4) - ecdf.ppf(1/4), **kwargs) <= true_iqr) == 1 for _ in tqdm(range(COVERAGE_PROB_ITERS))]))
    print('For Entropy:', np.mean([np.sum(nonparam_bootstrapping(RV, emp_entropy, **kwargs) <= true_entropy) == 1 for _ in tqdm(range(COVERAGE_PROB_ITERS))]))

##### Normal(0, 1) Distribution

In [35]:
evaluate_all(norm)

*** BOOTSTRAP CONFIDENCE INTERVALS ***
For Mean: [-0.21189116  0.47400186]
For Median: [-0.5582355   0.20114741]
For IQR: [0.62655702 1.84173077]


/var/folders/mg/l90d6s9536n6mfzjbqgrxffm0000gn/T/ipykernel_14340/184846182.py:8: RuntimeWarning: divide by zero encountered in log
  return np.ma.masked_invalid(-np.log([probs[np.argmax(bin_edges >= x) - 1] for x in ecdf.cdf_func.x[1:]])).mean()


For Entropy: [0.95472345 1.4238737 ]

*** TRUE PARAMETER VALUES ***
For Mean: 0.0
For Median: 0.0
For IQR: 1.3489795003921634
For Entropy: 1.4189385332046727

*** COVERAGE PROBABILITIES ***







































































































100%|██████████| 100/100 [02:35<00:00,  1.56s/it]


For Mean: 0.88


...

For Median: 0.89







































































































100%|██████████| 100/100 [02:41<00:00,  1.61s/it]


For IQR: 0.99







































































































100%|██████████| 100/100 [02:35<00:00,  1.55s/it]

For Entropy: 0.25


##### Normal(5, 1) Distribution

In [36]:
evaluate_all(norm, loc=5)

*** BOOTSTRAP CONFIDENCE INTERVALS ***
For Mean: [4.54407847 5.86331687]
For Median: [5.19154826 5.58311259]
For IQR: [0.43862485 2.50004286]


/var/folders/mg/l90d6s9536n6mfzjbqgrxffm0000gn/T/ipykernel_14340/184846182.py:8: RuntimeWarning: divide by zero encountered in log
  return np.ma.masked_invalid(-np.log([probs[np.argmax(bin_edges >= x) - 1] for x in ecdf.cdf_func.x[1:]])).mean()


For Entropy: [0.69228039 1.20404541]

*** TRUE PARAMETER VALUES ***
For Mean: 5.0
For Median: 5.0
For IQR: 1.3489795003921632
For Entropy: 1.4189385332046727

*** COVERAGE PROBABILITIES ***


















 75%|███████▌  | 75/100 [23:37<07:52, 18.90s/it]





















































































100%|██████████| 100/100 [02:33<00:00,  1.53s/it]


For Mean: 0.9


...

For Median: 0.89


100%|██████████| 100/100 [02:38<00:00,  1.59s/it]


For IQR: 0.99


100%|██████████| 100/100 [02:33<00:00,  1.53s/it]

For Entropy: 0.21


##### Normal(5, 9) Distribution

In [37]:
evaluate_all(norm, loc=5, scale=3)

*** BOOTSTRAP CONFIDENCE INTERVALS ***
For Mean: [2.90109661 4.83245616]
For Median: [3.54515963 7.51430283]
For IQR: [1.93222055 6.94140671]


/var/folders/mg/l90d6s9536n6mfzjbqgrxffm0000gn/T/ipykernel_14340/184846182.py:8: RuntimeWarning: divide by zero encountered in log
  return np.ma.masked_invalid(-np.log([probs[np.argmax(bin_edges >= x) - 1] for x in ecdf.cdf_func.x[1:]])).mean()


For Entropy: [2.03881469 2.43126985]

*** TRUE PARAMETER VALUES ***
For Mean: 5.0
For Median: 5.0
For IQR: 4.046938501176491
For Entropy: 2.5175508218727822

*** COVERAGE PROBABILITIES ***


100%|██████████| 100/100 [02:37<00:00,  1.58s/it]


For Mean: 0.91


...

For Median: 0.88


100%|██████████| 100/100 [02:44<00:00,  1.64s/it]


For IQR: 0.97


100%|██████████| 100/100 [02:38<00:00,  1.58s/it]

For Entropy: 0.23


##### Cauchy(5, 9) Distribution

In [38]:
evaluate_all(cauchy, loc=5, scale=3)

*** BOOTSTRAP CONFIDENCE INTERVALS ***
For Mean: [ 2.57912799 21.31019478]
For Median: [3.89168833 8.7548592 ]
For IQR: [ 2.86052313 20.96148041]


/var/folders/mg/l90d6s9536n6mfzjbqgrxffm0000gn/T/ipykernel_14340/184846182.py:8: RuntimeWarning: divide by zero encountered in log
  return np.ma.masked_invalid(-np.log([probs[np.argmax(bin_edges >= x) - 1] for x in ecdf.cdf_func.x[1:]])).mean()


For Entropy: [2.32940523 4.34997666]

*** TRUE PARAMETER VALUES ***
For Mean: nan
For Median: 5.0
For IQR: 6.0
For Entropy: 3.6296365356374007

*** COVERAGE PROBABILITIES ***


100%|██████████| 100/100 [02:47<00:00,  1.67s/it]


For Mean: 0.0


...

For Median: 0.9


100%|██████████| 100/100 [02:54<00:00,  1.74s/it]


For IQR: 0.96


100%|██████████| 100/100 [02:45<00:00,  1.66s/it]

For Entropy: 0.36


##### 1/2 * Normal(5, 1) + 1/2 * Cauchy(10, 1) Distribution

In [39]:
evaluate_all(CustomCDFRV(lambda x: (norm.cdf(x, loc=5, scale=1) + cauchy.cdf(x, loc=10, scale=1))/2))

*** BOOTSTRAP CONFIDENCE INTERVALS ***
For Mean: [ 7.44585701 11.71170629]
For Median: [4.95979594 9.36897526]
For IQR: [4.02626404 6.73763352]


/var/folders/mg/l90d6s9536n6mfzjbqgrxffm0000gn/T/ipykernel_14340/184846182.py:8: RuntimeWarning: divide by zero encountered in log
  return np.ma.masked_invalid(-np.log([probs[np.argmax(bin_edges >= x) - 1] for x in ecdf.cdf_func.x[1:]])).mean()


For Entropy: [1.8393712 2.6346163]

*** TRUE PARAMETER VALUES ***
For Mean: 7.500000000000184
For Median: 6.368752841046282
For IQR: 5.153524566844071
For Entropy: 2.6268575248912187

*** COVERAGE PROBABILITIES ***


100%|██████████| 100/100 [02:41<00:00,  1.61s/it]


For Mean: 0.91


...

For Median: 0.91


100%|██████████| 100/100 [02:48<00:00,  1.68s/it]


For IQR: 0.94


100%|██████████| 100/100 [02:41<00:00,  1.61s/it]

For Entropy: 0.39


#### Smoothed Bootstrap

I used the Scott's Rule of Thumb to choose the bandwidth i.e. 
$$h = 3.49\times s_n\times n^{\frac{-1}{3}}$$
and use Gaussian Kernel in our Kernel Density Estimator for the pdf i.e.
$$\hat{f}_{KDE}(x) = \frac{1}{nh}\sum_{i=1}^n K(\frac{x - X_i}{h})$$
where $\{X_i\}_{i=1}^n$ is our sample,
$$K(y) = \frac{1}{2\pi}\exp{-\frac{y^2}{2}}$$

The resampled data $\{X_i^*\}_{i=1}^n \sim \hat{f}_{KDE}$. Since, the estimator for the pdf is available in this case we don't use histogram method in this case.

In [75]:
def bw_scott(data: np.ndarray):
    std_dev = np.std(data)
    n = len(data)
    return 3.49 * std_dev * n ** (-0.333)

def gaussian_kernel(u):
    return 1 / np.sqrt(2 * np.pi) * np.exp(-1 / 2 * u * u)

class KDE:
    
    def __init__(self, data):
        self.data = data
        self.h = bw_scott(data)
    
    def eval(self, x):
        if np.isscalar(x):
            return np.mean(gaussian_kernel((x - self.data) / self.h)) / self.h
        result = np.zeros_like(x)
        for i in range(len(result)):
            result[i] = np.mean(gaussian_kernel((x[i] - self.data) / self.h)) / self.h
        return result

In [76]:
def smoothed_bootstrapping(RV, statistic, sample_size: int = SAMPLE_SIZE, num_bootstraps: int = NUM_BOOTSTRAPS, resample_statistic = None, **kwargs):

    # get the sample
    sample_data = RV.rvs(size=sample_size, **kwargs)

    # gets the KDE for the sample
    net_ecdf_rv = CustomPDFRV(KDE(sample_data).eval, np.min(sample_data), np.max(sample_data))
    
    # get point estimate
    point_estimate = statistic(net_ecdf_rv)

    # gets the resampled data
    resampled_data = np.array([net_ecdf_rv.rvs(size=sample_size) for _ in range(num_bootstraps)])

    # gets the ECDF of each resampled data
    # ecdf_data = [CustomPDFRV(KDE(bootstrap_sample).eval) for bootstrap_sample in resampled_data] # --> Taking too much time
    ecdf_data = [CustomCDFRV(ECDF(bootstrap_sample), np.min(bootstrap_sample), np.max(bootstrap_sample)) for bootstrap_sample in resampled_data]

    # gets the resample data statistic
    if resample_statistic is None:
        resampled_statistics = np.array([np.sqrt(sample_size) * (statistic(ecdf) - point_estimate) for ecdf in ecdf_data])
    else:
        resampled_statistics = np.array([np.sqrt(sample_size) * (resample_statistic(ecdf) - point_estimate) for ecdf in ecdf_data])

    # gets the resampled data statistic's ecdf
    resampled_statistic_ecdf = CustomCDFRV(ECDF(resampled_statistics))

    return np.array([point_estimate + resampled_statistic_ecdf.ppf(0.025)/np.sqrt(sample_size), point_estimate + 
            resampled_statistic_ecdf.ppf(0.975)/np.sqrt(sample_size)])

In [77]:
def evaluate_all(RV, **kwargs):
    print('*** BOOTSTRAP CONFIDENCE INTERVALS ***')
    print('For Mean:', smoothed_bootstrapping(RV, lambda ecdf: ecdf.mean(), resample_statistic= lambda ecdf: ecdf.cdf_func.x[1:].mean(), **kwargs))
    print('For Median:', smoothed_bootstrapping(RV, lambda ecdf: ecdf.median(), **kwargs))
    print('For IQR:', smoothed_bootstrapping(RV, lambda ecdf: ecdf.ppf(3/4) - ecdf.ppf(1/4), **kwargs))
    print('For Entropy:', smoothed_bootstrapping(RV, lambda ecdf: ecdf.entropy(), resample_statistic=emp_entropy, **kwargs))

    true_mean = RV.mean(**kwargs)
    true_median = RV.median(**kwargs)
    true_iqr = RV.ppf(3/4, **kwargs) - RV.ppf(1/4, **kwargs)
    true_entropy = RV.entropy(**kwargs)

    print()
    print('*** TRUE PARAMETER VALUES ***')
    print('For Mean:', true_mean)
    print('For Median:', true_median)
    print('For IQR:', true_iqr)
    print('For Entropy:', true_entropy)

    print()
    print('*** COVERAGE PROBABILITIES ***')
    print('For Mean:', np.mean([np.sum(smoothed_bootstrapping(RV, lambda ecdf: ecdf.mean(), resample_statistic= lambda ecdf: ecdf.cdf_func.x[1:].mean(), **kwargs) <= true_mean) == 1 for _ in tqdm(range(COVERAGE_PROB_ITERS))]))
    print('For Median:', np.mean([np.sum(smoothed_bootstrapping(RV, lambda ecdf: ecdf.median(), **kwargs) <= true_median) == 1 for _ in tqdm(range(COVERAGE_PROB_ITERS))]))
    print('For IQR:', np.mean([np.sum(smoothed_bootstrapping(RV, lambda ecdf: ecdf.ppf(3/4) - ecdf.ppf(1/4), **kwargs) <= true_iqr) == 1 for _ in tqdm(range(COVERAGE_PROB_ITERS))]))
    print('For Entropy:', np.mean([np.sum(smoothed_bootstrapping(RV, lambda ecdf: ecdf.entropy(), resample_statistic=emp_entropy, **kwargs) <= true_entropy) == 1 for _ in tqdm(range(COVERAGE_PROB_ITERS))]))

##### Normal(0, 1) Distribution

In [78]:
evaluate_all(norm)

*** BOOTSTRAP CONFIDENCE INTERVALS ***
For Mean: [0.37956001 1.44032119]
For Median: [-0.7182745   1.72252339]


/var/folders/mg/l90d6s9536n6mfzjbqgrxffm0000gn/T/ipykernel_14340/184846182.py:8: RuntimeWarning: divide by zero encountered in log
  return np.ma.masked_invalid(-np.log([probs[np.argmax(bin_edges >= x) - 1] for x in ecdf.cdf_func.x[1:]])).mean()


For IQR: [1.31442342 3.03179664]
For Entropy: [1.0932311  1.47212776]

*** TRUE PARAMETER VALUES ***
For Mean: 0.0
For Median: 0.0
For IQR: 1.3489795003921634
For Entropy: 1.4189385332046727

*** COVERAGE PROBABILITIES ***


100%|██████████| 100/100 [07:58<00:00,  4.78s/it]


For Mean: 0.51


...

For Median: 0.68


100%|██████████| 100/100 [04:18<00:00,  2.58s/it]


For IQR: 0.76


100%|██████████| 100/100 [04:33<00:00,  2.73s/it]

For Entropy: 0.22


##### Normal(5, 1) Distribution

In [79]:
evaluate_all(norm, loc=5)

*** BOOTSTRAP CONFIDENCE INTERVALS ***
For Mean: [4.9678375  6.12508014]
For Median: [4.28781672 6.58786756]
For IQR: [1.39739952 2.8337963 ]


/var/folders/mg/l90d6s9536n6mfzjbqgrxffm0000gn/T/ipykernel_14340/184846182.py:8: RuntimeWarning: divide by zero encountered in log
  return np.ma.masked_invalid(-np.log([probs[np.argmax(bin_edges >= x) - 1] for x in ecdf.cdf_func.x[1:]])).mean()


For Entropy: [0.07039372 0.93735559]

*** TRUE PARAMETER VALUES ***
For Mean: 5.0
For Median: 5.0
For IQR: 1.3489795003921632
For Entropy: 1.4189385332046727

*** COVERAGE PROBABILITIES ***


100%|██████████| 100/100 [07:28<00:00,  4.49s/it]


For Mean: 0.54


...

For Median: 0.7


100%|██████████| 100/100 [04:10<00:00,  2.51s/it]


For IQR: 0.66


100%|██████████| 100/100 [04:35<00:00,  2.76s/it]

For Entropy: 0.21


##### Normal(5, 9) Distribution

In [80]:
evaluate_all(norm, loc=5, scale=3)

*** BOOTSTRAP CONFIDENCE INTERVALS ***
For Mean: [ 7.55407692 11.74544434]
For Median: [3.45838119 8.2923204 ]
For IQR: [ 4.60340286 12.94796089]


/var/folders/mg/l90d6s9536n6mfzjbqgrxffm0000gn/T/ipykernel_14340/184846182.py:8: RuntimeWarning: divide by zero encountered in log
  return np.ma.masked_invalid(-np.log([probs[np.argmax(bin_edges >= x) - 1] for x in ecdf.cdf_func.x[1:]])).mean()


For Entropy: [1.83713415 2.24751639]

*** TRUE PARAMETER VALUES ***
For Mean: 5.0
For Median: 5.0
For IQR: 4.046938501176491
For Entropy: 2.5175508218727822

*** COVERAGE PROBABILITIES ***


100%|██████████| 100/100 [07:46<00:00,  4.66s/it]


For Mean: 0.57


...

For Median: 0.84


100%|██████████| 100/100 [04:18<00:00,  2.58s/it]


For IQR: 0.79


100%|██████████| 100/100 [04:44<00:00,  2.84s/it]

For Entropy: 0.27


##### Cauchy(5, 9) Distribution

In [82]:
evaluate_all(cauchy, loc=5, scale=3)

*** BOOTSTRAP CONFIDENCE INTERVALS ***
For Mean: [ 6.01155396 11.16804623]
For Median: [ 4.09035856 26.41916247]
For IQR: [128.86722781 314.50691924]


/var/folders/mg/l90d6s9536n6mfzjbqgrxffm0000gn/T/ipykernel_14340/184846182.py:8: RuntimeWarning: divide by zero encountered in log
  return np.ma.masked_invalid(-np.log([probs[np.argmax(bin_edges >= x) - 1] for x in ecdf.cdf_func.x[1:]])).mean()


For Entropy: [4.16669986 4.4567505 ]

*** TRUE PARAMETER VALUES ***
For Mean: nan
For Median: 5.0
For IQR: 6.0
For Entropy: 3.6296365356374007

*** COVERAGE PROBABILITIES ***


KeyboardInterrupt: 

##### 1/2 * Normal(5, 1) + 1/2 * Cauchy(10, 1) Distribution

In [ ]:
evaluate_all(CustomCDFRV(lambda x: (norm.cdf(x, loc=5, scale=1) + cauchy.cdf(x, loc=10, scale=1))/2))

*** BOOTSTRAP CONFIDENCE INTERVALS ***
For Mean: [ 9.93029769 13.5951231 ]
For Median: [ 2.02599997 10.42763695]
For IQR: [2.97169496 7.22987251]

*** TRUE PARAMETER VALUES ***
For Mean: 7.500000000000184
For Median: 6.368752841046282
For IQR: 5.153524566844071


#### Parametric Bootstrap

Here, the estimator of the $F$ is taken to be coming from a parametric family which is assumed to the same family as given in the assignment questions. In this case, after obtaining a sample $\{X_i\}_{i=1}^n\sim F$. I employ MLE to find the parameters of the distribution family which is assumed in that case eg. for $\mathcal{N}(5, 9)$, the family of distribution assumed is $\mathcal{N}(\mu, \sigma^2)$ and we estimate $\mu$ and $\sigma^2$ from the sample. Then after we have got the estimator of $F$ with the parameters, its pdf is also readily available and thus we don't use histogram here for Entropy calculation.

The MLE estimates are found computationally rather than using a closed form formulae as there is also a mixture distribution whose parameter needs to be estimated.

In [83]:
def param_bootstrapping(RV, statistic, sample_size: int = SAMPLE_SIZE, num_bootstraps: int = NUM_BOOTSTRAPS, fscale=None, resample_statistic = None, **kwargs):

    # get the sample
    sample_data = RV.rvs(size=sample_size, **kwargs)

    # fit the dist to the data
    if fscale is None:
        fit_params = RV.fit(sample_data)
    else:
        fit_params = RV.fit(sample_data, fscale=fscale)

    # fitted cdf
    fit_rv = CustomCDFRV(lambda x: RV.cdf(x, loc=fit_params[0], scale=fit_params[1]))

    # get point estimate
    point_estimate = statistic(fit_rv)

    # gets the resampled data
    resampled_data = np.array([fit_rv.rvs(size=sample_size) for _ in range(num_bootstraps)])

    # gets the ECDF of each resampled data
    ecdf_data = [CustomCDFRV(ECDF(bootstrap_sample), np.min(bootstrap_sample), np.max(bootstrap_sample)) for bootstrap_sample in resampled_data]

    # gets the resample data statistic
    if resample_statistic is None:
        resampled_statistics = np.array([np.sqrt(sample_size) * (statistic(ecdf) - point_estimate) for ecdf in ecdf_data])
    else:
        resampled_statistics = np.array([np.sqrt(sample_size) * (resample_statistic(ecdf) - point_estimate) for ecdf in ecdf_data])

    # gets the resampled data statistic's ecdf
    resampled_statistic_ecdf = CustomCDFRV(ECDF(resampled_statistics))

    return np.array([point_estimate + resampled_statistic_ecdf.ppf(0.025)/np.sqrt(sample_size), point_estimate + 
            resampled_statistic_ecdf.ppf(0.975)/np.sqrt(sample_size)])

In [86]:
def evaluate_all(RV, fs=None, **kwargs):
    print('*** BOOTSTRAP CONFIDENCE INTERVALS ***')
    print('For Mean:', param_bootstrapping(RV, lambda ecdf: ecdf.mean(), resample_statistic= lambda ecdf: ecdf.cdf_func.x[1:].mean(), fscale=fs, **kwargs))
    print('For Median:', param_bootstrapping(RV, lambda ecdf: ecdf.median(), fscale=fs, **kwargs))
    print('For IQR:', param_bootstrapping(RV, lambda ecdf: ecdf.ppf(3/4) - ecdf.ppf(1/4), fscale=fs, **kwargs))
    print('For Entropy:', param_bootstrapping(RV, lambda ecdf: ecdf.entropy(), resample_statistic=emp_entropy, fscale=fs, **kwargs))

    true_mean = RV.mean(**kwargs)
    true_median = RV.median(**kwargs)
    true_iqr = RV.ppf(3/4, **kwargs) - RV.ppf(1/4, **kwargs)
    true_entropy = RV.entropy(**kwargs)

    print()
    print('*** TRUE PARAMETER VALUES ***')
    print('For Mean:', true_mean)
    print('For Median:', true_median)
    print('For IQR:', true_iqr)
    print('For Entropy:', true_entropy)

    print()
    print('*** COVERAGE PROBABILITIES ***')
    print('For Mean:', np.mean([np.sum(param_bootstrapping(RV, lambda ecdf: ecdf.mean(), resample_statistic= lambda ecdf: ecdf.cdf_func.x[1:].mean(), fscale=fs, **kwargs) <= true_mean) == 1 for _ in tqdm(range(COVERAGE_PROB_ITERS))]))
    print('For Median:', np.mean([np.sum(param_bootstrapping(RV, lambda ecdf: ecdf.median(), fscale=fs, **kwargs) <= true_median) == 1 for _ in tqdm(range(COVERAGE_PROB_ITERS))]))
    print('For IQR:', np.mean([np.sum(param_bootstrapping(RV, lambda ecdf: ecdf.ppf(3/4) - ecdf.ppf(1/4), fscale=fs, **kwargs) <= true_iqr) == 1 for _ in tqdm(range(COVERAGE_PROB_ITERS))]))
    print('For Entropy:', np.mean([np.sum(param_bootstrapping(RV, lambda ecdf: ecdf.entropy(), resample_statistic=emp_entropy, fscale=fs, **kwargs) <= true_entropy) == 1 for _ in tqdm(range(COVERAGE_PROB_ITERS))]))

##### Normal(5, 1) Distribution

In [87]:
evaluate_all(norm, loc=5, fs=1)

*** BOOTSTRAP CONFIDENCE INTERVALS ***
For Mean: [4.55784072 5.35014026]
For Median: [4.25652549 5.30979888]
For IQR: [0.82725383 2.00752362]


/var/folders/mg/l90d6s9536n6mfzjbqgrxffm0000gn/T/ipykernel_14340/184846182.py:8: RuntimeWarning: divide by zero encountered in log
  return np.ma.masked_invalid(-np.log([probs[np.argmax(bin_edges >= x) - 1] for x in ecdf.cdf_func.x[1:]])).mean()


For Entropy: [0.79120456 1.49710595]

*** TRUE PARAMETER VALUES ***
For Mean: 5.0
For Median: 5.0
For IQR: 1.3489795003921632
For Entropy: 1.4189385332046727

*** COVERAGE PROBABILITIES ***












































































































































































































100%|██████████| 100/100 [01:24<00:00,  1.18it/s]


For Mean: 0.92


...

For Median: 0.95


100%|██████████| 100/100 [01:21<00:00,  1.22it/s]


For IQR: 1.0


100%|██████████| 100/100 [01:26<00:00,  1.16it/s]

For Entropy: 0.92


##### Normal(5, 9) Distribution

In [88]:
evaluate_all(norm, loc=5, scale=3)

*** BOOTSTRAP CONFIDENCE INTERVALS ***
For Mean: [3.42448174 5.67431746]
For Median: [4.79231939 7.52415556]
For IQR: [1.47959081 4.58257527]


/var/folders/mg/l90d6s9536n6mfzjbqgrxffm0000gn/T/ipykernel_14340/184846182.py:8: RuntimeWarning: divide by zero encountered in log
  return np.ma.masked_invalid(-np.log([probs[np.argmax(bin_edges >= x) - 1] for x in ecdf.cdf_func.x[1:]])).mean()


For Entropy: [2.03785279 2.62053982]

*** TRUE PARAMETER VALUES ***
For Mean: 5.0
For Median: 5.0
For IQR: 4.046938501176491
For Entropy: 2.5175508218727822

*** COVERAGE PROBABILITIES ***


100%|██████████| 100/100 [01:07<00:00,  1.47it/s]


For Mean: 0.9


...

For Median: 0.93


100%|██████████| 100/100 [01:08<00:00,  1.45it/s]


For IQR: 0.96


100%|██████████| 100/100 [01:13<00:00,  1.37it/s]

For Entropy: 0.56


##### Cauchy(5, 9) Distribution

In [89]:
evaluate_all(cauchy, loc=5, scale=3)

*** BOOTSTRAP CONFIDENCE INTERVALS ***
For Mean: [-31.75345636  19.02842995]
For Median: [5.17784845 8.08320285]
For IQR: [ 3.60236731 14.51614878]


/var/folders/mg/l90d6s9536n6mfzjbqgrxffm0000gn/T/ipykernel_14340/184846182.py:8: RuntimeWarning: divide by zero encountered in log
  return np.ma.masked_invalid(-np.log([probs[np.argmax(bin_edges >= x) - 1] for x in ecdf.cdf_func.x[1:]])).mean()


For Entropy: [2.5298373  3.57294999]

*** TRUE PARAMETER VALUES ***
For Mean: nan
For Median: 5.0
For IQR: 6.0
For Entropy: 3.6296365356374007

*** COVERAGE PROBABILITIES ***


100%|██████████| 100/100 [01:11<00:00,  1.41it/s]


For Mean: 0.0


...

For Median: 0.94


100%|██████████| 100/100 [01:11<00:00,  1.41it/s]


For IQR: 0.96


100%|██████████| 100/100 [01:15<00:00,  1.32it/s]

For Entropy: 0.56


##### 1/2 * Normal(5, 1) + 1/2 * Cauchy(10, 1) Distribution

In [90]:
class MixNormCauchy(stats.rv_continuous):
    def __init__(self, xtol = 1e-14, seed = None):
        super().__init__(xtol=xtol, seed=seed)

    def _cdf(self, x, *args, **kwds):
        args, loc, scale = self._parse_args(*args, **kwds)
        x, loc, scale = map(np.asarray, (x, loc, scale))
        args = tuple(map(np.asarray, args))
        return (norm.cdf(x, loc=loc) + cauchy.cdf(x, loc=2*loc))/2
    
    # def monte_mean(self, num_monte_carlo:int = 100): # it is faster than finding mean by integration
    #     return np.mean(self.rvs(size=num_monte_carlo))
    
    # def _entropy(self, num_monte_carlo:int = 100): # doing integration gives unstable result
    #     return np.mean(-np.log(self._pdf(self.rvs(size=num_monte_carlo))))

In [91]:
cust_mix_rv = MixNormCauchy()

In [92]:
evaluate_all(cust_mix_rv, loc=5, fs=1)

*** BOOTSTRAP CONFIDENCE INTERVALS ***


/Users/rishideychowdhury/Desktop/M.STAT. 1ST YEAR/ASSIGNMENTS/RESAMPLING-TECHNIQUES/env/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:2041: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  return integrate.quad(self._mom_integ1, 0, 1, args=(m,)+args)[0]
/Users/rishideychowdhury/Desktop/M.STAT. 1ST YEAR/ASSIGNMENTS/RESAMPLING-TECHNIQUES/env/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:2788: RuntimeWarning: invalid value encountered in sqrt
  Shat = sqrt(mu2hat / mu2)


For Mean: [ 2.59602667 13.60920197]
For Median: [4.36592086 5.76353503]
For IQR: [0.90459326 2.96649989]


/var/folders/mg/l90d6s9536n6mfzjbqgrxffm0000gn/T/ipykernel_14340/184846182.py:8: RuntimeWarning: divide by zero encountered in log
  return np.ma.masked_invalid(-np.log([probs[np.argmax(bin_edges >= x) - 1] for x in ecdf.cdf_func.x[1:]])).mean()


For Entropy: [1.13437559 2.10100139]

*** TRUE PARAMETER VALUES ***
For Mean: 4.99999999999999
For Median: 5.0
For IQR: 1.5879546167017224
For Entropy: -inf

*** COVERAGE PROBABILITIES ***


...

For Median: 0.96


100%|██████████| 100/100 [04:30<00:00,  2.71s/it]


For IQR: 1.0


100%|██████████| 100/100 [04:42<00:00,  2.83s/it]

For Entropy: 0.0


#### Jackknife

I used leave $3$ out Jackknife here.

In [101]:
def emp_entropy(sample_data):
    iqr = np.quantile(sample_data, 3/4, method='lower') - np.quantile(sample_data, 1/4, method='lower')
    h = 2 * iqr * np.float_power(len(sample_data), -1/3)
    num_bins = int((np.max(sample_data) - np.min(sample_data)) // h)
    probs, bin_edges = np.histogram(sample_data, density=True, bins=num_bins)
    bin_edges =  np.concatenate([[-np.inf], bin_edges, [np.inf]])
    probs = np.concatenate([[0.0], probs, [0.0]])
    return np.ma.masked_invalid(-np.log([probs[np.argmax(bin_edges >= x) - 1] for x in sample_data])).mean()

In [102]:
def d_jackknife(RV, statistic, sample_size: int = SAMPLE_SIZE, d = 3, **kwargs):

    # get the sample
    sample_data = RV.rvs(size=sample_size, **kwargs)

    # get full sample point estimate
    point_estimate = statistic(sample_data)

    # gets the resampled data i.e. leave d out sample data
    resampled_data = np.array([np.delete(sample_data, idxs) for idxs in itertools.combinations(list(range(len(sample_data))), d)])

    # get the resampled data point estimates
    resampled_point_estimates = np.array(
        [np.sqrt(sample_size * (sample_size-d) / d) * (statistic(res_dat) - point_estimate) for res_dat in resampled_data])

    # gets the resampled data statistic's ecdf
    resampled_statistic_ecdf = CustomCDFRV(ECDF(resampled_point_estimates))

    return np.array([point_estimate + resampled_statistic_ecdf.ppf(0.025)/np.sqrt(sample_size), point_estimate + 
            resampled_statistic_ecdf.ppf(0.975)/np.sqrt(sample_size)])

In [103]:
def evaluate_all(RV, **kwargs):
    print('*** BOOTSTRAP CONFIDENCE INTERVALS ***')
    print('For Mean:', d_jackknife(RV, lambda data: data.mean(), **kwargs))
    print('For Median:', d_jackknife(RV, lambda data: np.quantile(data, 1/2, method='lower'), **kwargs))
    print('For IQR:', d_jackknife(RV, lambda data: np.quantile(data, 3/4, method='lower') - np.quantile(data, 1/4, method='lower'), **kwargs))
    print('For Entropy:', d_jackknife(RV, emp_entropy, **kwargs))

    true_mean = RV.mean(**kwargs)
    true_median = RV.median(**kwargs)
    true_iqr = RV.ppf(3/4, **kwargs) - RV.ppf(1/4, **kwargs)
    true_entropy = RV.entropy(**kwargs)

    print()
    print('*** TRUE PARAMETER VALUES ***')
    print('For Mean:', true_mean)
    print('For Median:', true_median)
    print('For IQR:', true_iqr)
    print('For Entropy:', true_entropy)

    print()
    print('*** COVERAGE PROBABILITIES ***')
    print('For Mean:', np.mean([np.sum(d_jackknife(RV, lambda data: data.mean(), **kwargs) <= true_mean) == 1 for _ in tqdm(range(COVERAGE_PROB_ITERS))]))
    print('For Median:', np.mean([np.sum(d_jackknife(RV, lambda data: np.quantile(data, 1/2, method='lower'), **kwargs) <= true_median) == 1 for _ in tqdm(range(COVERAGE_PROB_ITERS))]))
    print('For IQR:', np.mean([np.sum(d_jackknife(RV, lambda data: np.quantile(data, 3/4, method='lower') - np.quantile(data, 1/4, method='lower'), **kwargs) <= true_iqr) == 1 for _ in tqdm(range(COVERAGE_PROB_ITERS))]))
    print('For Entropy:', np.mean([np.sum(d_jackknife(norm, emp_entropy, **kwargs) <= true_entropy) == 1 for _ in tqdm(range(COVERAGE_PROB_ITERS))]))

##### Normal(0, 1) Distribution

In [104]:
evaluate_all(norm)

*** BOOTSTRAP CONFIDENCE INTERVALS ***
For Mean: [-0.11050515  0.94982404]
For Median: [-0.20374528  0.04447268]
For IQR: [1.08843927 3.53965701]


/var/folders/mg/l90d6s9536n6mfzjbqgrxffm0000gn/T/ipykernel_14340/2537570582.py:8: RuntimeWarning: divide by zero encountered in log
  return np.ma.masked_invalid(-np.log([probs[np.argmax(bin_edges >= x) - 1] for x in sample_data])).mean()


For Entropy: [0.19923662 1.24437761]

*** TRUE PARAMETER VALUES ***
For Mean: 0.0
For Median: 0.0
For IQR: 1.3489795003921634
For Entropy: 1.4189385332046727

*** COVERAGE PROBABILITIES ***


100%|██████████| 100/100 [00:00<00:00, 100.48it/s]


For Mean: 0.97


...

For Median: 0.7


100%|██████████| 100/100 [00:04<00:00, 21.81it/s]


For IQR: 0.82


100%|██████████| 100/100 [00:15<00:00,  6.48it/s]

For Entropy: 0.35


##### Normal(5, 1) Distribution

In [105]:
evaluate_all(norm, loc=5)

*** BOOTSTRAP CONFIDENCE INTERVALS ***
For Mean: [4.76069716 5.48552068]
For Median: [5.10458089 5.64537977]
For IQR: [0.82292554 2.92503926]


/var/folders/mg/l90d6s9536n6mfzjbqgrxffm0000gn/T/ipykernel_14340/2537570582.py:8: RuntimeWarning: divide by zero encountered in log
  return np.ma.masked_invalid(-np.log([probs[np.argmax(bin_edges >= x) - 1] for x in sample_data])).mean()


For Entropy: [0.82201923 1.4371317 ]

*** TRUE PARAMETER VALUES ***
For Mean: 5.0
For Median: 5.0
For IQR: 1.3489795003921632
For Entropy: 1.4189385332046727

*** COVERAGE PROBABILITIES ***


100%|██████████| 100/100 [00:00<00:00, 100.79it/s]


For Mean: 0.92


...

For Median: 0.76


100%|██████████| 100/100 [00:04<00:00, 21.62it/s]


For IQR: 0.75


100%|██████████| 100/100 [00:15<00:00,  6.55it/s]

For Entropy: 0.32


##### Normal(5, 9) Distribution

In [106]:
evaluate_all(norm, loc=5, scale=3)

/var/folders/mg/l90d6s9536n6mfzjbqgrxffm0000gn/T/ipykernel_14340/2537570582.py:8: RuntimeWarning: divide by zero encountered in log
  return np.ma.masked_invalid(-np.log([probs[np.argmax(bin_edges >= x) - 1] for x in sample_data])).mean()


*** BOOTSTRAP CONFIDENCE INTERVALS ***
For Mean: [4.68493257 7.28574526]
For Median: [-0.89287618  7.91335492]
For IQR: [1.31207675 4.64403241]
For Entropy: [1.68328412 2.11231588]

*** TRUE PARAMETER VALUES ***
For Mean: 5.0
For Median: 5.0
For IQR: 4.046938501176491
For Entropy: 2.5175508218727822

*** COVERAGE PROBABILITIES ***


100%|██████████| 100/100 [00:00<00:00, 100.60it/s]


For Mean: 0.94


100%|██████████| 100/100 [00:02<00:00, 35.94it/s]


For Median: 0.76


100%|██████████| 100/100 [00:04<00:00, 21.88it/s]


For IQR: 0.76


100%|██████████| 100/100 [00:15<00:00,  6.55it/s]

For Entropy: 0.35


##### Cauchy(5, 9) Distribution

In [107]:
evaluate_all(cauchy, loc=5, scale=3)

/var/folders/mg/l90d6s9536n6mfzjbqgrxffm0000gn/T/ipykernel_14340/2537570582.py:8: RuntimeWarning: divide by zero encountered in log
  return np.ma.masked_invalid(-np.log([probs[np.argmax(bin_edges >= x) - 1] for x in sample_data])).mean()


*** BOOTSTRAP CONFIDENCE INTERVALS ***
For Mean: [0.80157654 4.92630639]
For Median: [3.14922966 6.58503558]
For IQR: [-0.95228441  4.76771224]
For Entropy: [2.42204078 3.54312769]

*** TRUE PARAMETER VALUES ***
For Mean: nan
For Median: 5.0
For IQR: 6.0
For Entropy: 3.6296365356374007

*** COVERAGE PROBABILITIES ***


100%|██████████| 100/100 [00:00<00:00, 101.59it/s]


For Mean: 0.0


100%|██████████| 100/100 [00:02<00:00, 36.74it/s]


For Median: 0.75


100%|██████████| 100/100 [00:04<00:00, 21.73it/s]


For IQR: 0.76


100%|██████████| 100/100 [00:15<00:00,  6.55it/s]

For Entropy: 0.0


##### 1/2 * Normal(5, 1) + 1/2 * Cauchy(10, 1) Distribution

In [108]:
evaluate_all(CustomCDFRV(lambda x: (norm.cdf(x, loc=5, scale=1) + cauchy.cdf(x, loc=10, scale=1))/2))

/var/folders/mg/l90d6s9536n6mfzjbqgrxffm0000gn/T/ipykernel_14340/2537570582.py:8: RuntimeWarning: divide by zero encountered in log
  return np.ma.masked_invalid(-np.log([probs[np.argmax(bin_edges >= x) - 1] for x in sample_data])).mean()


*** BOOTSTRAP CONFIDENCE INTERVALS ***
For Mean: [4.99979314 7.14813206]
For Median: [5.19237486 6.73772502]
For IQR: [3.91109829 6.51280264]
For Entropy: [1.20193834 2.73690367]

*** TRUE PARAMETER VALUES ***
For Mean: 7.500000000000184
For Median: 6.368752841046282
For IQR: 5.153524566844071
For Entropy: 2.4161299833945207

*** COVERAGE PROBABILITIES ***


100%|██████████| 100/100 [00:03<00:00, 30.37it/s]


For Mean: 0.95


100%|██████████| 100/100 [00:05<00:00, 19.66it/s]


For Median: 0.72


100%|██████████| 100/100 [00:06<00:00, 14.42it/s]


For IQR: 0.78


100%|██████████| 100/100 [00:15<00:00,  6.53it/s]

For Entropy: 0.0


#### Asymptotic Theory

The following results are applicable for the first $3$ family of distributions.

For mean, we use the sample mean as the estimator for the population mean and the asymptotic distribution is as follows,
$$\frac{\sqrt{n}(\bar{X}_n - \mu)}{s_n} \xrightarrow{d} \mathcal{N}(0, 1)$$

For median, we use the sample median as the estimator for the population median and the asymptotic distribution is as follows,
$$2\sqrt{n}f(\xi_{\frac{1}{2}})\cdot(F_n^{-1}(\frac{1}{2}) - \xi_{\frac{1}{2}})\xrightarrow{d} \mathcal{N}(0, 1)$$

For IQR, we use the sample IQR as the estimator of the population IQR and the asymptotic distribution is as follows,
$$
\sqrt{n}\left(\mathrm{IQR} - \left(\xi_{\frac{3}{4}}-\xi_{\frac{1}{4}}\right)\right)\xrightarrow{d} \mathcal{N}\left(0, \frac{1}{16}\left[\frac{3}{f^{2}(\xi_{\frac{3}{4}})}+\frac{3}{f^{2}(\xi_{\frac{1}{4}})}-\frac{2}{f(\xi_{\frac{1}{4}})f(\xi_{\frac{3}{4}})}\right]\right)
$$

I use the histogram as seen above as a estimator for density $f$.

In [137]:
def hist_emp_pdf(sample_data, x):
    iqr = np.quantile(sample_data, 3/4, method='lower') - np.quantile(sample_data, 1/4, method='lower')
    h = 2 * iqr * np.float_power(len(sample_data), -1/3)
    num_bins = int((np.max(sample_data) - np.min(sample_data)) // h)
    probs, bin_edges = np.histogram(sample_data, density=True, bins=num_bins)
    bin_edges =  np.concatenate([[-np.inf], bin_edges, [np.inf]])
    probs = np.concatenate([[0.0], probs, [0.0]])
    return probs[np.argmax(bin_edges >= x) - 1]

In [138]:
def get_sample(RV, sample_size, **kwargs):
    
    # get the sample
    sample_data = RV.rvs(size=sample_size, **kwargs)

    return sample_data

def get_conf_int(RV, mode = 'mean', sample_size=SAMPLE_SIZE, **kwargs):

    # get the sample
    sample_data = RV.rvs(size=sample_size, **kwargs)
    
    # point estimate
    if mode == 'mean':
        point_estimate = sample_data.mean()
        std = np.std(sample_data, ddof=1)
    
    if mode == 'median':
        point_estimate = np.quantile(sample_data, 1/2, method='lower')
        std = 1/(2 * hist_emp_pdf(sample_data, point_estimate))

    if mode == 'iqr':
        point_estimate = np.quantile(sample_data, 3/4, method='lower') - np.quantile(sample_data, 1/4, method='lower')
        std = (3 / (hist_emp_pdf(sample_data, np.quantile(sample_data, 3/4, method='lower'))**2)
               + 3 / (hist_emp_pdf(sample_data, np.quantile(sample_data, 1/4, method='lower'))**2)
               - 2 / (hist_emp_pdf(sample_data, np.quantile(sample_data, 3/4, method='lower')) * hist_emp_pdf(sample_data, np.quantile(sample_data, 1/4, method='lower')))) / 4
    

    return np.array([point_estimate - 1.96 * std / np.sqrt(sample_data.shape[0]), point_estimate + 1.96 * std / np.sqrt(sample_data.shape[0])])

In [139]:
def evaluate_all(RV, **kwargs):
    print('*** BOOTSTRAP CONFIDENCE INTERVALS ***')
    print('For Mean:', get_conf_int(RV, 'mean', **kwargs))
    print('For Median:', get_conf_int(RV, 'median', **kwargs))
    print('For IQR:', get_conf_int(RV, 'iqr', **kwargs))
    # print('For Entropy:', d_jackknife(RV, emp_entropy, **kwargs))

    true_mean = RV.mean(**kwargs)
    true_median = RV.median(**kwargs)
    true_iqr = RV.ppf(3/4, **kwargs) - RV.ppf(1/4, **kwargs)
    # true_entropy = RV.entropy(**kwargs)

    print()
    print('*** TRUE PARAMETER VALUES ***')
    print('For Mean:', true_mean)
    print('For Median:', true_median)
    print('For IQR:', true_iqr)
    # print('For Entropy:', true_entropy)

    print()
    print('*** COVERAGE PROBABILITIES ***')
    print('For Mean:', np.mean([np.sum(get_conf_int(RV, 'mean', **kwargs) <= true_mean) == 1 for _ in range(COVERAGE_PROB_ITERS)]))
    print('For Median:', np.mean([np.sum(get_conf_int(RV, 'median', **kwargs) <= true_median) == 1 for _ in range(COVERAGE_PROB_ITERS)]))
    print('For IQR:', np.mean([np.sum(get_conf_int(RV, 'iqr', **kwargs) <= true_iqr) == 1 for _ in range(COVERAGE_PROB_ITERS)]))
    # print('For Entropy:', np.mean([np.sum(d_jackknife(norm, emp_entropy, **kwargs) <= true_entropy) == 1 for _ in tqdm(range(COVERAGE_PROB_ITERS))]))

##### Normal(0, 1) Distribution

In [140]:
evaluate_all(norm)

*** BOOTSTRAP CONFIDENCE INTERVALS ***
For Mean: [-0.372475    0.57972623]
For Median: [-0.81476458  0.44249041]
For IQR: [-3.31413579  7.10199332]

*** TRUE PARAMETER VALUES ***
For Mean: 0.0
For Median: 0.0
For IQR: 1.3489795003921634

*** COVERAGE PROBABILITIES ***
For Mean: 0.95
For Median: 0.94
For IQR: 1.0


##### Normal(5, 1) Distribution

In [141]:
evaluate_all(norm, loc=5)

*** BOOTSTRAP CONFIDENCE INTERVALS ***
For Mean: [4.5818142  5.43587679]
For Median: [4.76140365 5.73253021]
For IQR: [-5.26806398  8.69147649]

*** TRUE PARAMETER VALUES ***
For Mean: 5.0
For Median: 5.0
For IQR: 1.3489795003921632

*** COVERAGE PROBABILITIES ***
For Mean: 0.91
For Median: 0.9
For IQR: 1.0


##### Normal(5, 9) Distribution

In [142]:
evaluate_all(norm, loc=5, scale=3)

*** BOOTSTRAP CONFIDENCE INTERVALS ***
For Mean: [3.42211374 5.94054789]
For Median: [2.39853997 6.22680545]
For IQR: [-44.78629159  57.34472519]

*** TRUE PARAMETER VALUES ***
For Mean: 5.0
For Median: 5.0
For IQR: 4.046938501176491

*** COVERAGE PROBABILITIES ***
For Mean: 0.98
For Median: 0.94
For IQR: 1.0


### Summary of the Results and Findings

The following are the expected confidence intervals.

## Expected Confidence Interval

#### Standard Normal Distribution

| Resampling Technique     | Mean                    | Median                  | IQR                     | Entropy                   |
|--------------------------|-------------------------|-------------------------|-------------------------|---------------------------|
| Exact Resampling         | [-0.50162051, 0.46058489] | [-0.44724029, 0.48643302] | [0.85025879, 1.92980969] | [0.90920166, 1.42997591]  |
| Non-Parametric Bootstrapping | [-0.21189116, 0.47400186] | [-0.5582355, 0.20114741] | [0.62655702, 1.84173077] | [0.95472345, 1.4238737]   |
| Smoothed Bootstrapping   | [0.37956001, 1.44032119] | [-0.7182745, 1.72252339] | [1.31442342, 3.03179664] | [1.0932311, 1.47212776]   |
| Parametric Bootstrapping | [NA, NA] | [4.25652549, 5.30979888] | [0.82725383, 2.00752362] | [0.79120456, 1.49710595]  |
| d-Jackknife              | [-0.11050515, 0.94982404] | [-0.20374528, 0.04447268] | [1.08843927, 3.53965701] | [0.19923662, 1.24437761]  |
| Asymptotic Theory        | [-0.372475, 0.57972623] | [-0.81476458, 0.44249041] | [-3.31413579, 7.10199332] | [NA, NA]                  |


#### Normal(5, 1) Distribution

| Resampling Technique     | Mean                     | Median                   | IQR                      | Entropy                    |
|--------------------------|--------------------------|--------------------------|--------------------------|----------------------------|
| Exact Resampling         | [4.53342066, 5.34385121] | [4.66192238, 5.54904764] | [0.78117713, 1.9739037]  | [0.81874155, 1.4221989]    |
| Non-Parametric Bootstrapping | [4.54407847, 5.86331687] | [5.19154826, 5.58311259] | [0.43862485, 2.50004286] | [0.69228039, 1.20404541]   |
| Smoothed Bootstrapping   | [4.9678375, 6.12508014]  | [4.28781672, 6.58786756] | [1.39739952, 2.8337963]  | [0.07039372, 0.93735559]   |
| Parametric Bootstrapping | [4.55784072, 5.35014026] | [4.25652549, 5.30979888] | [0.82725383, 2.00752362] | [0.79120456, 1.49710595]   |
| d-Jackknife              | [4.76069716, 5.48552068] | [5.10458089, 5.64537977] | [0.82292554, 2.92503926] | [0.82201923, 1.4371317]    |
| Asymptotic Theory        | [4.5818142, 5.43587679]  | [4.76140365, 5.73253021] | [-5.26806398, 8.69147649] | [NA, NA]                   |


#### Normal(5, 9) Distribution

| Resampling Technique     | Mean                     | Median                   | IQR                      | Entropy                    |
|--------------------------|--------------------------|--------------------------|--------------------------|----------------------------|
| Exact Resampling         | [3.64363594, 6.30702352] | [3.89787655, 7.07393481] | [2.08583044, 5.20921586] | [1.86699938, 2.61216594]   |
| Non-Parametric Bootstrapping | [2.90109661, 4.83245616] | [3.54515963, 7.51430283] | [1.93222055, 6.94140671] | [2.03881469, 2.43126985]   |
| Smoothed Bootstrapping   | [7.55407692, 11.74544434] | [3.45838119, 8.2923204]  | [4.60340286, 12.94796089] | [1.83713415, 2.24751639]   |
| Parametric Bootstrapping | [3.42448174, 5.67431746] | [4.79231939, 7.52415556] | [1.47959081, 4.58257527] | [2.03785279, 2.62053982]   |
| d-Jackknife              | [4.68493257, 7.28574526] | [-0.89287618, 7.91335492] | [1.31207675, 4.64403241] | [1.68328412, 2.11231588]   |
| Asymptotic Theory        | [3.42211374, 5.94054789] | [2.39853997, 6.22680545] | [-44.78629159, 57.34472519] | [NA, NA]                   |


#### Cauchy(5, 9) Distribution

| Resampling Technique     | Mean                     | Median                   | IQR                      | Entropy                    |
|--------------------------|--------------------------|--------------------------|--------------------------|----------------------------|
| Exact Resampling         | [-25.43186907, 44.29733189] | [3.05560127, 8.69550191] | [3.28674209, 14.15490763] | [2.35118421, 3.84739899]   |
| Non-Parametric Bootstrapping | [2.57912799, 21.31019478] | [3.89168833, 8.7548592]  | [2.86052313, 20.96148041] | [2.32940523, 4.34997666]   |
| Smoothed Bootstrapping   | [6.01155396, 11.16804623] | [4.09035856, 26.41916247] | [128.86722781, 314.50691924] | [4.16669986, 4.4567505]    |
| Parametric Bootstrapping | [-31.75345636, 19.02842995] | [5.17784845, 8.08320285] | [3.60236731, 14.51614878] | [2.5298373, 3.57294999]    |
| d-Jackknife              | [0.80157654, 4.92630639] | [3.14922966, 6.58503558] | [-0.95228441, 4.76771224] | [2.42204078, 3.54312769]   |


##### 1/2 * Normal(5, 1) + 1/2 * Cauchy(10, 1) Distribution

| Resampling Technique     | Mean                     | Median                   | IQR                      | Entropy                    |
|--------------------------|--------------------------|--------------------------|--------------------------|----------------------------|
| Exact Resampling         | [3.5957454, 14.10338629] | [5.2445834, 9.79953697]  | [2.11966964, 6.47262403] | [1.86530187, 2.69646429]   |
| Non-Parametric Bootstrapping | [7.44585701, 11.71170629] | [4.95979594, 9.36897526] | [4.02626404, 6.73763352] | [1.8393712, 2.6346163]     |
| Smoothed Bootstrapping   | [9.93029769, 13.5951231] | [2.02599997, 10.42763695] | [2.97169496, 7.22987251] | [1.1231342, 2.6356163]     |
| Parametric Bootstrapping | [2.59602667, 13.60920197] | [4.36592086, 5.76353503] | [0.90459326, 2.96649989] | [1.13437559, 2.10100139]   |
| d-Jackknife              | [4.99979314, 7.14813206] | [5.19237486, 6.73772502] | [3.91109829, 6.51280264] | [1.20193834, 2.73690367]   |


## Expected Interval Lengths

Here's a comparison of Monte Carlo expected interval lengths for each resampling method across various distributions:

#### Standard Normal Distribution
| Resampling Technique     | Mean Interval Length    | Median Interval Length  | IQR Interval Length     | Entropy Interval Length |
|--------------------------|-------------------------|-------------------------|-------------------------|-------------------------|
| Exact Resampling         | 0.96220540              | 0.93367331              | 1.07955090              | 0.52077425              |
| Non-Parametric Bootstrapping | 0.68589302              | 0.75938291              | 1.21517375              | 0.46915025              |
| Smoothed Bootstrapping   | 1.06076118              | 2.44079789              | 1.71737322              | 0.37889666              |
| d-Jackknife              | 1.06032919              | 0.24821795              | 2.45121774              | 1.04514099              |
| Asymptotic Theory        | 0.95220123              | 1.25725599              | 10.41612911             | NA                      |

#### Normal(5, 1) Distribution
| Resampling Technique     | Mean Interval Length    | Median Interval Length  | IQR Interval Length     | Entropy Interval Length |
|--------------------------|-------------------------|-------------------------|-------------------------|-------------------------|
| Exact Resampling         | 0.81043055              | 0.88712526              | 1.19272657              | 0.60345735              |
| Non-Parametric Bootstrapping | 1.31923840              | 0.39256433              | 2.06141801              | 0.51176402              |
| Smoothed Bootstrapping   | 1.15724264              | 2.30005084              | 1.43639678              | 0.86696187              |
| Parametric Bootstrapping | 0.79229954              | 0.10527339              | 1.18026979              | 0.70595039              |
| d-Jackknife              | 0.72482352              | 0.54079888              | 2.10211372              | 0.61411147              |
| Asymptotic Theory        | 0.85406259              | 0.97112656              | 14.95954047             | NA                      |

#### Normal(5, 9) Distribution
| Resampling Technique     | Mean Interval Length    | Median Interval Length  | IQR Interval Length     | Entropy Interval Length |
|--------------------------|-------------------------|-------------------------|-------------------------|-------------------------|
| Exact Resampling         | 2.66338758              | 3.17605825              | 3.12338542              | 0.74516656              |
| Non-Parametric Bootstrapping | 1.93135955              | 3.96914320              | 5.00918616              | 0.39245516              |
| Smoothed Bootstrapping   | 4.19136742              | 4.83393921              | 8.34455803              | 0.41038224              |
| Parametric Bootstrapping | 2.24983572              | 2.73183617              | 3.10298446              | 0.58268703              |
| d-Jackknife              | 2.60081269              | 8.8062311               | 3.33195566              | 0.42903176              |
| Asymptotic Theory        | 2.51843415              | 3.82826548              | 102.06500878            | NA                      |

#### Cauchy(5, 9) Distribution
| Resampling Technique     | Mean Interval Length    | Median Interval Length  | IQR Interval Length     | Entropy Interval Length |
|--------------------------|-------------------------|-------------------------|-------------------------|-------------------------|
| Exact Resampling         | 69.72960196             | 5.63990064              | 10.86816555             | 1.49621478              |
| Non-Parametric Bootstrapping | 18.73106679             | 4.86317087              | 18.10095728             | 2.02057143              |
| Smoothed Bootstrapping   | 5.15649227              | 22.32880391             | 185.63934543            | 0.29005064              |
| Parametric Bootstrapping | 50.39088631             | 2.90535440              | 10.91378147             | 1.04311269              |
| d-Jackknife              | 4.12472985              | 3.43580592              | 5.71999666              | 1.12108691              |

#### 1/2 * Normal(5, 1) + 1/2 * Cauchy(10, 1) Distribution
| Resampling Technique     | Mean Interval Length    | Median Interval Length  | IQR Interval Length     | Entropy Interval Length |
|--------------------------|-------------------------|-------------------------|-------------------------|-------------------------|
| Exact Resampling         | 10.50764089             | 3.44494643              | 4.35295439              | 0.83116242              |
| Non-Parametric Bootstrapping | 4.26584928              | 4.40918036              | 2.71136948              | 0.7952441               |
| Smoothed Bootstrapping   | 3.66482541              | 8.40163698              | 4.25817755              | 1.5124811               |
| Parametric Bootstrapping | 11.0131753              | 1.39761417              | 2.06190662              | 0.9666258               |
| d-Jackknife              | 2.14833892              | 1.54535016              | 2.60170435              | 1.53496533              |

## Coverage Probabilities

Below are the coverage probabilities,

#### Standard Normal Distribution

| Resampling Technique     | Mean | Median | IQR | Entropy |
|--------------------------|------|--------|-----|---------|
| Exact Resampling         | 1.0  | 1.0    | 1.0 | 0.92    |
| Non-Parametric Bootstrapping | 0.88 | 0.89   | 0.99| 0.25    |
| Smoothed Bootstrapping   | 0.51 | 0.68   | 0.76| 0.22    |
| d-Jackknife              | 0.97 | 0.7    | 0.82| 0.35    |
| Asymptotic Theory        | 0.95 | 0.94   | 1.0 | NA      |

#### Normal(5, 1) Distribution

| Resampling Technique     | Mean | Median | IQR | Entropy |
|--------------------------|------|--------|-----|---------|
| Exact Resampling         | 1.0  | 1.0    | 1.0 | 0.89    |
| Non-Parametric Bootstrapping | 0.9  | 0.89   | 0.99| 0.21    |
| Smoothed Bootstrapping   | 0.54 | 0.7    | 0.66| 0.21    |
| Parametric Bootstrapping | 0.92 | 0.95   | 1.0 | 0.92    |
| d-Jackknife              | 0.92 | 0.76   | 0.75| 0.32    |
| Asymptotic Theory        | 0.91 | 0.9    | 1.0 | NA      |

#### Normal(5, 9) Distribution

| Resampling Technique     | Mean | Median | IQR | Entropy |
|--------------------------|------|--------|-----|---------|
| Exact Resampling         | 1.0  | 1.0    | 1.0 | 0.9     |
| Non-Parametric Bootstrapping | 0.91 | 0.88   | 0.97| 0.23    |
| Smoothed Bootstrapping   | 0.57 | 0.84   | 0.79| 0.27    |
| Parametric Bootstrapping | 0.9  | 0.93   | 0.96| 0.56    |
| d-Jackknife              | 0.94 | 0.76   | 0.76| 0.35    |
| Asymptotic Theory        | 0.98 | 0.94   | 1.0 | NA      |

#### Cauchy(5, 9) Distribution

| Resampling Technique     | Mean | Median | IQR | Entropy |
|--------------------------|------|--------|-----|---------|
| Exact Resampling         | 0.0  | 1.0    | 1.0 | 0.79    |
| Non-Parametric Bootstrapping | 0.0  | 0.9    | 0.96| 0.36    |
| Smoothed Bootstrapping   | 0.0  | 0.91   | 0.97| 0.4     |
| Parametric Bootstrapping | 0.0  | 0.94   | 0.96| 0.56    |
| d-Jackknife              | 0.0  | 0.75   | 0.76| 0.0     |

#### 1/2 * Normal(5, 1) + 1/2 * Cauchy(10, 1) Distribution

| Resampling Technique     | Mean | Median | IQR | Entropy |
|--------------------------|------|--------|-----|---------|
| Exact Resampling         | 1.0  | 1.0    | 1.0 | 0.91    |
| Non-Parametric Bootstrapping | 0.91 | 0.91   | 0.94| 0.39    |
| Smoothed Bootstrapping   | 0.8  | 0.9    | 0.97| 0.4     |
| Parametric Bootstrapping | 1.0  | 0.96   | 1.0 | 0.0     |
| d-Jackknife              | 0.95 | 0.72   | 0.78| 0.0     |